In [1]:
import pandas as pd

import numpy as np

import warnings

import sklearn.preprocessing
import sklearn.linear_model

from numba.core.errors import NumbaDeprecationWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=NumbaDeprecationWarning)

In [2]:
df = pd.read_csv('../data/california_housing/california_housing.csv').drop(['Unnamed: 0'], axis=1)

#nombre de maisons à prendre en compte (20k c'est beaucoup trop)
# quelques condition pour nettoyer la base de données
df = df.loc[df['Population']<10000]
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Seulement San Francisco :
# Latitude > 37,2
# Latitude < 38,07
# Longitude > -122,5
# Longitude < -121,75

#df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
#df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

#df = df.sample(n=2000, random_state = 9).reset_index(drop=True)

X = df.iloc[:,0:8]
Y = df.iloc[:,9]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 9)
model.fit(X, Y)

SHAP = df.iloc[:,[10,11,12,13,14,15,16,17]]

In [ ]:
import antakia

dataset = antakia.Dataset(X, model = model, y=Y)
dataset.frac(0.1)

In [ ]:
atk = antakia.AntakIA(dataset, import_explanation = SHAP)
atk.startGUI()